### Previsão de vendas com Análise de Séries Temporais.



https://www.kaggle.com/ravi07bec/ravi-ts

In [ ]:
### Versão modificada Por: RODRIGO SOUZA

#### Meu Linkedin: https://www.linkedin.com/in/ssrodrigo

## Objetivo:

•	Análise por tipo de loja e análise correlacional da atividade das lojas.

•	Realize extensa análise da série temporal (decomposição sazonal, tendências, autocorrelação).

•	Prever as próximas 6 semanas de vendas usando Prophet (metodologia do Facebook).


In [ ]:
pip install prophet

In [ ]:
# Download and unzip our zipfile
from urllib.request import urlopen
from zipfile import ZipFile

zipurl = 'https://raw.githubusercontent.com/rajeevratan84/datascienceforbusiness/master/rossman_train.zip'
zipresp = urlopen(zipurl) # Create a new file on the hard drive
tempzip = open("/tmp/tempfile.zip", "wb") # Write the contents of the downloaded file into the new file
tempzip.write(zipresp.read()) # Close the newly-created file
tempzip.close() # Re-open the newly-created file with ZipFile()
zf = ZipFile("/tmp/tempfile.zip") # Extract its contents into <extraction_path>
zf.extractall(path = '') # note that extractall will automatically create the path, left blank so it's in working directory
# close the ZipFile instance
zf.close()

In [ ]:
import warnings
warnings.filterwarnings("ignore")

# loading packages
# basic + dates 
import numpy as np
import pandas as pd
from pandas import datetime

# data visualization
import matplotlib.pyplot as plt
import seaborn as sns # advanced vizs
%matplotlib inline

# statistics
from statsmodels.distributions.empirical_distribution import ECDF

# time series analysis
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf

# prophet by Facebook
from fbprophet import Prophet

In [ ]:
 from prophet import Prophet

In [ ]:
# Loading our Datasets
train = pd.read_csv("rossman_train.csv", parse_dates = True, low_memory = False, index_col = 'Date')

# Load the additional store data
file_name = "https://raw.githubusercontent.com/rajeevratan84/datascienceforbusiness/master/rossman_store.csv"
store = pd.read_csv(file_name, low_memory=False)

# time series as indexes
train.index

In [ ]:
# first glance at the train set: head and tail
print("In total: ", train.shape)
train.head(5)

In [ ]:
# data extraction
train['Year'] = train.index.year
train['Month'] = train.index.month
train['Day'] = train.index.day
train['WeekOfYear'] = train.index.weekofyear

# adding new variable
train['SalePerCustomer'] = train['Sales']/train['Customers']
train['SalePerCustomer'].describe()

In [ ]:
# To get the first impression about continious variables in the data we can plot ECDF.
sns.set(style = "ticks")# to format into seaborn 
c = '#386B7F' # basic color for plots
plt.figure(figsize = (12, 6))

plt.subplot(311)
cdf = ECDF(train['Sales'])
plt.plot(cdf.x, cdf.y, label = "statmodels", color = c);
plt.xlabel('Sales'); plt.ylabel('ECDF');

# plot second ECDF  
plt.subplot(312)
cdf = ECDF(train['Customers'])
plt.plot(cdf.x, cdf.y, label = "statmodels", color = c);
plt.xlabel('Customers');

# plot second ECDF  
plt.subplot(313)
cdf = ECDF(train['SalePerCustomer'])
plt.plot(cdf.x, cdf.y, label = "statmodels", color = c);
plt.xlabel('Sale per Customer');

In [ ]:
# closed stores
train[(train.Open == 0) & (train.Sales == 0)].head()

Há 172817 lojas fechadas nos dados. É cerca de 10% da quantidade total de observações. Para evitar previsões tendenciosas, vamos baixar esses valores.

Que tal abrir lojas sem vendas?

In [ ]:
zero_sales = train[(train.Open != 0) & (train.Sales == 0)]
print("In total: ", zero_sales.shape)
zero_sales.head(5)

In [ ]:
print("Closed stores and days which didn't have any sales won't be counted into the forecasts.")
train = train[(train["Open"] != 0) & (train['Sales'] != 0)]

print("In total: ", train.shape)

# additional information about the stores
store.head()

In [ ]:
# missing values?
store.isnull().sum()

In [ ]:
# missing values in CompetitionDistance
store[pd.isnull(store.CompetitionDistance)]

In [ ]:
# fill NaN with a median value (skewed distribuion)
store['CompetitionDistance'].fillna(store['CompetitionDistance'].median(), inplace = True)

In [ ]:
# no promo = no information about the promo?
_ = store[pd.isnull(store.Promo2SinceWeek)]
_[_.Promo2 != 0].shape

In [ ]:
# replace NA's by 0
store.fillna(0, inplace = True)

In [ ]:
print("Joining train set with an additional store information.")

# by specifying inner join we make sure that only those observations 
# that are present in both train and store sets are merged together
train_store = pd.merge(train, store, how = 'inner', on = 'Store')

print("In total: ", train_store.shape)
train_store.head()

In [ ]:
train_store.groupby('StoreType')['Sales'].describe()

O StoreType B tem a maior média de vendas entre todas as outras, porém temos muito menos dados para isso. Então vamos imprimir uma soma geral de vendas e clientes para ver qual StoreType é o mais vendido e lotado:

In [ ]:
train_store.groupby('StoreType')['Customers', 'Sales'].sum()

In [ ]:
train_store.head()

Claramente as lojas do tipo A. StoreType D vão em segundo lugar tanto em Vendas quanto em Clientes. E os períodos de encontro? A grade de facetas da Seaborn é a melhor ferramenta para esta tarefa:

In [ ]:
sns.factorplot(data = train_store, x = 'Month', y = "Sales", col = 'Promo',hue = 'StoreType') 

In [ ]:
sns.factorplot(data = train_store, x = 'Month', y = "Customers", col = 'Promo',hue = 'StoreType') 

In [ ]:
sns.factorplot(data = train_store, x = 'Month', y = "SalePerCustomer", col = 'Promo',hue = 'StoreType')

StoreType B como o mais vendido e realizador, na realidade não é verdade. O maior valor salepercustomer é observado na LojaType D, cerca de 12€ com Promo e 10€ sem. Quanto à StoreType A e C, é de cerca de 9€.

O baixo valor do SalePerCustomer para StoreType B descreve seu Carrinho de Compradores: há muitas pessoas que compram essencialmente para coisas "pequenas" (ou em pequena quantidade). Além disso, vimos que no geral este StoreType gerou a menor quantidade de vendas e clientes durante o período.



Para concluir nossa análise preliminar de dados, podemos adicionar variáveis descrevendo o período de tempo durante o qual a concorrência e a promoção foram abertas:Para concluir nossa análise preliminar de dados, podemos adicionar variáveis descrevendo o período de tempo durante o qual a concorrência e a promoção foram abertas:

In [ ]:
train_store['CompetitionOpen'] = 12 * (train_store.Year - train_store.CompetitionOpenSinceYear) + \
        (train_store.Month - train_store.CompetitionOpenSinceMonth)
    
# Promo open time
train_store['PromoOpen'] = 12 * (train_store.Year - train_store.Promo2SinceYear) + \
        (train_store.WeekOfYear - train_store.Promo2SinceWeek) / 4.0

# replace NA's by 0
train_store.fillna(0, inplace = True)

# average PromoOpen time and CompetitionOpen time per store type
train_store.loc[:, ['StoreType', 'Sales', 'Customers', 'PromoOpen', 'CompetitionOpen']].groupby('StoreType').mean()

In [ ]:
# Compute the correlation matrix 
# exclude 'Open' variable
corr_all = train_store.drop('Open', axis = 1).corr()

# Generate a mask for the upper triangle
mask = np.zeros_like(corr_all, dtype = np.bool)
mask[np.triu_indices_from(mask)] = True

# Set up the matplotlib figure
f, ax = plt.subplots(figsize = (11, 9))

# Draw the heatmap with the mask and correct aspect ratio
sns.heatmap(corr_all, mask = mask,
            square = True, linewidths = .5, ax = ax, cmap = "BuPu")      
plt.show()

Como mencionado anteriormente, temos uma forte correlação positiva entre a quantidade de Vendas e Clientes de uma loja. Observamos também uma correlação positiva entre o fato de que a loja teve uma promoção em execução (Promo igual a 1) e quantidade de Clientes.



No entanto, assim que a loja continua uma promoção consecutiva (Promo2 igual a 1) o número de Clientes e Vendas parece permanecer o mesmo ou mesmo diminuir, o que é descrito pela correlação negativa pálida no mapa de calor. A mesma correlação negativa é observada entre a presença da promoção na loja e o dia de uma semana


In [ ]:
# sale per customer trends
sns.factorplot(data = train_store, x = 'DayOfWeek', y = "Sales", col = 'Promo',hue = 'Promo2')

Há várias coisas aqui:

- Em caso de não promoção, tanto Promo quanto Promo2 são iguais a 0, as vendas tendem a atingir o pico no domingo (!). Embora devemos notar que storeType C não funciona aos domingos. Portanto, são principalmente dados do StoreType A, B e D.

- Pelo contrário, as lojas que executam a promoção tendem a aproveitar ao máximo as vendas na segunda-feira. Esse fato pode ser um bom indicador para as campanhas de marketing da Rossmann. A mesma tendência segue as lojas que têm promoção ao mesmo tempo (Promo e Promo2 são iguais a 1).

- O Promo2 por si só não parece estar correlacionado com qualquer alteração significativa no valor de Vendas. Isso também pode ser prooved pela área azul pálida no mapa de calor acima.

# Conclusão do EDA
- O StoreType mais vendido e lotado é A.
- A melhor loja "Venda por Cliente"Type D indica para o carrinho comprador mais alto. Para se beneficiar desse fato, Rossmann pode considerar propor uma maior variedade de seus produtos.
- Baixo salePerCustomer valor para StoreType B indica o fato de que as pessoas compram lá essencialmente para coisas "pequenas". Mesmo assim, embora este StoreType tenha gerado a menor quantidade de vendas e clientes durante todo o período, ele mostra um grande potencial.


# Análise série de tempo por tipo de loja
O que torna uma série temporal diferente de um problema de regressão regular?

- É dependente do tempo. A suposição básica de uma regressão linear que as observações são independentes não se mantém neste caso.
- Juntamente com uma tendência crescente ou decrescente, a maioria das séries temporadas tem alguma forma de tendências de sazonalidade, ou seja, variações específicas para um determinado período de tempo. Por exemplo, para as férias de Natal, que veremos neste conjunto de dados.

Construímos uma análise de séries temporariais em tipos de lojas em vez de lojas individuais. A principal vantagem dessa abordagem é sua simplicidade de apresentação e contação geral de diferentes tendências e sazonalidades no conjunto de dados.

Nesta seção, analisaremos os dados da série temporal: suas tendências, sesonidades e autocorrelação. Normalmente, ao final da análise, somos capazes de desenvolver um modelo ARIMA (Autoregression Integrated Moving Average), mas não será nosso foco principal hoje. Em vez disso, tentamos entender os dados, e só mais tarde chegar às previsões usando a metodologia do Profeta.

## Sazonalidade
Pegamos quatro lojas de tipos de lojas para representar seu grupo:
- Armazenar o número 2 para StoreType A
- Armazenar o número 85 para StoreType B,
- Armazenar o número 1 para StoreType C
- Armazene o número 13 para StoreType D.
Também faz sentido diminuir a coleta de dados de dias a semanas usando o método de resample para ver as tendências atuais com mais clareza.


Prophet Bug - https://darektidwell.com/typeerror-float-argument-must-be-a-string-or-a-number-not-period-facebook-prophet-and-pandas/

In [ ]:
# Prophet Bug - Prophet deregisters the Pandas converters in its code.
pd.plotting.register_matplotlib_converters()

# preparation: input should be float type
train['Sales'] = train['Sales'] * 1.0

# store types
sales_a = train[train.Store == 2]['Sales']
sales_b = train[train.Store == 85]['Sales'].sort_index(ascending = True) # solve the reverse order
sales_c = train[train.Store == 1]['Sales']
sales_d = train[train.Store == 13]['Sales']

f, (ax1, ax2, ax3, ax4) = plt.subplots(4, figsize = (12, 13))

# store types
sales_a.resample('W').sum().plot(ax = ax1)
sales_b.resample('W').sum().plot(ax = ax2)
sales_c.resample('W').sum().plot(color = c, ax = ax3)
sales_d.resample('W').sum().plot(color = c, ax = ax4)

Confira a presença de uma tendência em séries.

In [ ]:
f, (ax1, ax2, ax3, ax4) = plt.subplots(4, figsize = (12, 13))

# monthly
decomposition_a = seasonal_decompose(sales_a, model = 'additive', freq = 365)
decomposition_a.trend.plot(color = c, ax = ax1)

decomposition_b = seasonal_decompose(sales_b, model = 'additive', freq = 365)
decomposition_b.trend.plot(color = c, ax = ax2)

decomposition_c = seasonal_decompose(sales_c, model = 'additive', freq = 365)
decomposition_c.trend.plot(color = c, ax = ax3)



decomposition_d = seasonal_decompose(sales_d, model = 'additive', freq = 365)
decomposition_d.trend.plot(color = c, ax = ax4)

Overall sales seems to increase, however not for the StoreType C (a third from the top). Eventhough the StoreType A is the most selling store type in the dataset, it seems that it cab follow the same decresing trajectory as StoreType C did.

Autocorrelaion
The next step in ourtime series analysis is to review Autocorrelation Function (ACF) and Partial Autocorrelation Function (PACF) plots.

ACF is a measure of the correlation between the timeseries with a lagged version of itself. For instance at lag 5, ACF would compare series at time instant ‘t1’…’tn’ with series at instant ‘t1-5’…’tn-5’ (t1-5 and tn being end points).

PACF, on the other hand, measures the correlation between the timeseries with a lagged version of itself but after eliminating the variations explained by the intervening comparisons. Eg. at lag 5, it will check the correlation but remove the effects already explained by lags 1 to 4.

In [ ]:
plt.figure(figsize = (12, 8))

# acf and pacf for A
plt.subplot(421); plot_acf(sales_a, lags = 50, ax = plt.gca(), color = c)
plt.subplot(422); plot_pacf(sales_a, lags = 50, ax = plt.gca(), color = c)

# acf and pacf for B
plt.subplot(423); plot_acf(sales_b, lags = 50, ax = plt.gca(), color = c)
plt.subplot(424); plot_pacf(sales_b, lags = 50, ax = plt.gca(), color = c)

# acf and pacf for C
plt.subplot(425); plot_acf(sales_c, lags = 50, ax = plt.gca(), color = c)
plt.subplot(426); plot_pacf(sales_c, lags = 50, ax = plt.gca(), color = c)

# acf and pacf for D
plt.subplot(427); plot_acf(sales_d, lags = 50, ax = plt.gca(), color = c)
plt.subplot(428); plot_pacf(sales_d, lags = 50, ax = plt.gca(), color = c)



plt.show()

Podemos ler esses enredos horizontalmente. Cada par horizontal é para um 'StoreType', de A a D. Em geral, essas tramas estão mostrando a correlação da série consigo mesma, defasada por x time units correlação da série consigo mesma, defasada por unidades x tempo.

Há duas coisas comuns para cada par de parcelas: não randomnes da série temporal e alta lag-1 (que provavelmente precisará de uma ordem maior de d/D diferente).


Tipo A e tipo B: Ambos os tipos apresentam sazonalidades em certos atrasos. Para o tipo A, é cada 12ª observação com picos positivos nos lags 12 (s) e 24(2s) e assim por diante. Para o tipo B é uma tendência semanal com picos positivos nos lags de 7(s), 14(2s), 21(3s) e 28(4s).
Tipo C e tipo D: Parcelas desses dois tipos são mais complexas. Parece que cada observação está coorrelada às suas observações adjacentes.


# Análise da Série Time e Previsão com Profeta
Previsão para as próximas 6 semanas para a primeira loja!

A equipe do Núcleo de Ciência de Dados do Facebook publicou recentemente um novo procedimento para prever dados de séries temporéticas chamadas Prophet. Baseia-se em um modelo aditivo onde tendências não lineares se encaixam com sazonalidade anual e semanal, além de feriados. Ele permite a realização de previsões automatizadas que já são implementadas em R em escala no Python 3.


In [ ]:
# Loading our Datasets
df = pd.read_csv("rossman_train.csv", parse_dates = True, low_memory = False, index_col = 'Date')

# remove closed stores and those with no sales
df = df[(df["Open"] != 0) & (df['Sales'] != 0)]

# sales for the store number 1 (StoreType C)
sales = df[df.Store == 1].loc[:, ['Date', 'Sales']]

# reverse to the order: from 2013 to 2015
sales = sales.sort_index(ascending = False)

# to datetime64
sales['Date'] = pd.DatetimeIndex(sales['Date'])
sales.dtypes

In [ ]:
sales = sales.rename(columns = {'Date': 'ds',
                                'Sales': 'y'})
sales = sales.reset_index()
del sales['ds']
sales = sales.rename(columns = {'Date': 'ds'})
sales

##Tratativa dos feriados

In [ ]:
# create holidays dataframe
state_dates  = df[(df.StateHoliday == 'a') | (df.StateHoliday == 'b') & (df.StateHoliday == 'c')].reset_index().loc[:, 'Date'].values
school_dates = df[df.SchoolHoliday == 1].reset_index().loc[:, 'Date'].values

state = pd.DataFrame({'holiday': 'state_holiday',
                      'ds': pd.to_datetime(state_dates)})
school = pd.DataFrame({'holiday': 'school_holiday',
                      'ds': pd.to_datetime(school_dates)})

holidays = pd.concat((state, school))      
holidays.head()

Execute nosso modelo de previsão (leva ~1 min)

In [ ]:
# set the uncertainty interval to 95% (the Prophet default is 80%)
my_model = Prophet(interval_width = 0.95, 
                   holidays = holidays)
my_model.fit(sales)

# dataframe that extends into future 6 weeks 
future_dates = my_model.make_future_dataframe(periods = 6*7)

print("First week to forecast.")
future_dates.tail(7)

In [ ]:
# predictions - takes around ~ 2 min to run
forecast = my_model.predict(future_dates)

# preditions for last week
forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].tail(7)

O objeto de previsão aqui é um novo dataframe que inclui uma coluna com a previsão, bem como colunas para componentes e intervalos de incerteza.

In [ ]:
fc = forecast[['ds', 'yhat']].rename(columns = {'Date': 'ds', 'Forecast': 'yhat'})

Prophet traça os valores observados de nossa série temporal (os pontos negros), os valores previstos (linha azul) e os intervalos de incerteza de nossas previsões (as regiões sombreadas azuis).

In [ ]:
my_model.plot(forecast);

One other particularly strong feature of Prophet is its ability to return the components of our forecasts. This can help reveal how daily, weekly and yearly patterns of the time series plus manyally included holidayes contribute to the overall forecasted values:

In [ ]:
my_model.plot_components(forecast);

O primeiro enredo mostra que as vendas mensais da loja número 1 vem diminuindo linearmente ao longo do tempo e a segunda mostra as lacunas holiays incluídas no modelo. A terceira parcela destaca o fato de que o volume semanal de vendas da semana passada chega para a segunda-feira da próxima semana, enquanto o próximo enredo mostra que a estação mais turbulenta ocorre durante as festas de Natal.